### data analysis process
Question$\Rightarrow$Wrangle(1.data aquisition; 2. data cleaning)$\Rightarrow$Explore(1. build intuition;2. find patterns$\Rightarrow$Draw conclusion(or make prediction)$\Rightarrow$Communication

#### csv——comma separaed values
like a spreadsheet with no formulas
easy to process with code

In [35]:
# unicodecsv 与 csv 的工作原理完全相同，但它是 Anaconda 附带的模块，并且支持 unicode
# 读取csv文件
import unicodecsv

def read_csv(filename):
    with open(filename,'rb') as f:
        reader = unicodecsv.DictReader(f)
        return list(reader)

In [36]:
enrollments = read_csv('enrollments.csv')
daily_engagement = read_csv('daily-engagement.csv')
project_submissions = read_csv('project-submissions.csv')

### 修正数据格式
读取的csv中所有数据都是string，不可以用来比较和相加，需要改为python中可用的形式

In [38]:
# take time as a string and return python datetime object
# if there is no date given, return none

from datetime import datetime as dt

def parse_date(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d')

In [39]:
def parse_maybe_int(i):
    if i == '':
        return None
    else:
        return int(i)

In [40]:
# clean up the data type in the enrollments table
# don't need to change account_key
# 选取某一列的时候也要用[]
for enrollment in enrollments:
    enrollment['cancel_date'] = parse_date(enrollment['cancel_date'])
    enrollment['days_to_cancel'] = parse_maybe_int(enrollment['days_to_cancel'])
    enrollment['is_canceled'] = enrollment['is_canceled'] == 'True'
    enrollment['is_udacity'] = enrollment['is_udacity'] == 'True'
    enrollment['join_date'] = parse_date(enrollment['join_date'])
# no need to write a function to change boolean value, 
# just check whether the string equals to 'True' and return a boolean value

In [43]:
# clean up the data type in the daily_engagement table
for engagement in daily_engagement:
    engagement['lessons_completed'] = int(float(engagement['lessons_completed']))
    engagement['num_courses_visited'] = int(float(engagement['num_courses_visited']))
    engagement['projects_completed'] = int(float(engagement['projects_completed']))
    engagement['total_minutes_visited'] = float(engagement['total_minutes_visited'])
    engagement['utc_date'] = parse_date(engagement['utc_date'])

In [45]:
#clean up the data type in project_submissions
for submission in project_submissions:
    submission['completion_date'] = parse_date(submission['completion_date'])
    submission['creation_date'] = parse_date(submission['creation_date'])

### 保持各表中数据名一致

In [52]:
# the account key in daily_engagement is named as acct
# modify the column name
for engagement_record in daily_engagement:
    engagement_record['account_key']=engagement_record['acct']
    del[engagement_record['acct']]

### 解决关于数据的疑问
统计数据

In [54]:
# find the total number of unique record
# elements in set is unrepeatable
def get_unique_students(datalist):
    unique_students = set()
    for data in datalist:
        unique_students.add(data['account_key'])
    return unique_students

In [55]:
print(len(enrollments))
unique_enrolled_students=get_unique_students(enrollments)
print(len(unique_enrolled_students))

1640
1302


In [56]:
print(len(daily_engagement))
unique_engagement_students=get_unique_students(daily_engagement)
print(len(unique_engagement_students))

136240
1237


In [57]:
print(len(project_submissions))
unique_project_submissers=get_unique_students(project_submissions)
print(len(unique_project_submissers))

3642
743


### Investigate problem
$\color{blue}{Why\ are\ students\ missing\ from\ daily engagement?}$
- 1. Indentify surprising data points
- - any enrollment record with no corresponding engagement data 找到engagements和enrollments中不对应的点
- 2. Print out one or a few surprisingdata points
- 3. fix problem
- - more investigation

In [60]:
# use elements in enrollment notin unique_enrolled_students, we can see more imformation of problem data point.
for enrollment in enrollments:
    if enrollment['account_key'] not in unique_engagement_students:
        print enrollment
        break
#  Python 的 break 语句，可以帮你打印单一的问题记录

{u'status': u'canceled', u'is_udacity': False, u'is_canceled': True, u'join_date': datetime.datetime(2014, 11, 12, 0, 0), u'account_key': u'1219', u'cancel_date': datetime.datetime(2014, 11, 12, 0, 0), u'days_to_cancel': 0}
